## 随机森林(RandomForest)

参考资料:

http://blog.csdn.net/qq547276542/article/details/78304454

https://www.toutiao.com/a6455335677880959502/

https://www.cnblogs.com/fionacai/p/5894142.html

### 概述

随机森林属于集成学习（Ensemble Learning）中的bagging算法。在集成学习中，主要分为bagging算法和boosting算法。我们先看看这两种方法的特点和区别。

** Bagging（套袋法）**

bagging的算法过程如下：

* 从原始样本集中使用Bootstraping方法随机抽取n个训练样本，共进行k轮抽取，得到k个训练集。（k个训练集之间相互独立，元素可以有重复）
* 对于k个训练集，我们训练k个模型（这k个模型可以根据具体问题而定，比如决策树，knn等）
* 对于分类问题：由投票表决产生分类结果；对于回归问题：由k个模型预测结果的均值作为最后预测结果。（所有模型的重要性相同）

** Boosting（提升法）**
boosting的算法过程如下：

* 对于训练集中的每个样本建立权值wi，表示对每个样本的关注度。当某个样本被误分类的概率很高时，需要加大对该样本的权值。
* 进行迭代的过程中，每一步迭代都是一个弱分类器。我们需要用某种策略将其组合，作为最终模型。（例如AdaBoost给每个弱分类器一个权值，将其线性组合最为最终分类器。误差越小的弱分类器，权值越大）

** Bagging，Boosting的主要区别 **

* 样本选择上：Bagging采用的是Bootstrap随机有放回抽样；而Boosting每一轮的训练集是不变的，改变的只是每一个样本的权重。
* 样本权重：Bagging使用的是均匀取样，每个样本权重相等；Boosting根据错误率调整样本权重，错误率越大的样本权重越大。
* 预测函数：Bagging所有的预测函数的权重相等；Boosting中误差越小的预测函数其权重越大。
* 并行计算：Bagging各个预测函数可以并行生成；Boosting各个预测函数必须按顺序迭代生成。

下面是将决策树与这些算法框架进行结合所得到的新的算法：

1）Bagging + 决策树 = 随机森林

2）AdaBoost + 决策树 = 提升树

3）Gradient Boosting + 决策树 = GBDT

这块的资料可以参考:
http://blog.csdn.net/code_lr/article/details/51446492

http://blog.csdn.net/qq547276542/article/details/78304454

### 原理

决策树提供了一个简单、清晰的概念模型来理解迭代分类过程。然而，在实践中，单个决策树对于解决涉及大量变量和中等大小规模数据的现实问题时，并不十分有效。因此，我们需要重型武器——由一群“专家”组成的决策森林。

这里的“专家”指的是什么呢？

我们认为的“专家”们，每人脑子里有一棵决策树模型，然后将这样的很多个专家集合在一起，不那么严格地讲，我们就创建了一个随机森林。当我们期望决策结果好于单个决策树的时候，我们会想要一群“专家”来帮忙做决策。因此，我们需要一些方法来决定如何整理和排序“专家”们的“意见”。

当然为了让决策意见更多样化,所以在构建这些专家时,使用随机样本,随机特征,这样构建的专家就会更具差异性,意见也就更丰富了 决策效果就更好

### 优缺点

** 随机森林有许多优点：**

* 具有极高的准确率
* 随机性的引入，使得随机森林不容易过拟合
* 随机性的引入，使得随机森林有很好的抗噪声能力
* 能处理很高维度的数据，并且不用做特征选择
* 既能处理离散型数据，也能处理连续型数据，数据集无需规范化
* 训练速度快，可以得到变量重要性排序
* 容易实现并行化

** 随机森林的缺点：**

* 当随机森林中的决策树个数很多时，训练时需要的空间和时间会较大
* 随机森林模型还有许多不好解释的地方，有点算个黑盒模型

### 构建过程

* 通过自助法(bootstrap)构建大小为n的一个训练集，即重复抽样选择n个训练样例
* 对于刚才新得到的训练集，构建一棵决策树。在每个节点执行以下操作：
* 通过不重复抽样选择d个特征
* 利用上面的d个特征，选择某种度量分割节点
* 重复步骤1和2，k次
* 对于每一个测试样例，对k颗决策树的预测结果进行投票。票数最多的结果就是随机森林的预测结果。

随机森林中构建决策树的做法和原始决策树的区别是，在每次分割节点时，不是从所有特征中选择而是在一个小特征集中选择特征。

虽然随机森林模型的可解释性不如决策树，但是它的一大优点是受超参数的影响波动不是很大(译者注：几个主要参数还是需要好好调参的)。我们也不需要对随机森林进行剪枝因为集成模型的鲁棒性很强，不会过多受单棵决策树噪音的影响。


在实际运用随机森林模型时，树的数目(k)需要好好调参。一般，k越大，随机森林的性能越好，当然计算成本也越高。

样本大小n能够控制bias-variance平衡，如果n很大，我们就减小了随机性因此随机森林就容易过拟合。另一方面，如果n很小，虽然不会过拟合，但模型的性能会降低。大多数随机森林的实现，包括sklearn中的RandomForestClassifier，n的大小等于原始训练集的大小。


在每一次分割时特征集的大小d，一个最起码的要求是要小于原始特征集大小，sklearn中的默认值，其中m是原始特征集大小，这是一个比较合理的数值。

### 实例

我们使用鸢尾花数据集作为测试数据集

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import numpy as np

In [13]:
iris=load_iris()
X_train,X_test,y_train,y_test=train_test_split(iris.data,iris.target,test_size=0.3,random_state=0)

载入随机森林模型 其中n_estimators表示树的个数  criterion表示分割特征的方式 默认gini 可选entropy     max_depth最大深度

In [14]:
forest=RandomForestClassifier(n_estimators=10,random_state=0)

In [15]:
forest.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [17]:
y_pred=forest.predict(X_test)

打印混淆矩阵让预测结果更加直观

In [19]:
confusion_matrix(y_test,y_pred)

array([[16,  0,  0],
       [ 0, 17,  1],
       [ 0,  0, 11]])

In [22]:
forest.score(X_test,y_test)

0.97777777777777775

我们还可以通过随机森林做特征选择

我们可以通过随机森林中所有决策树得到**平均不纯度衰减**来度量特征的重要性,sklearn中就是训练后模型的 feature_importances_ 参数

In [23]:
forest.feature_importances_

array([ 0.14334485,  0.0264803 ,  0.40058315,  0.42959169])

In [25]:
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

通过不纯度衰减我们可以知道  
* petal width 对于分类是最重要的特征  
* sepal width特征对于分类是最不重要的 

我们现在把最不重要的特征去掉 再重新训练看会不会影响结果

In [31]:
X_train[0:5]

array([[ 5. ,  2. ,  3.5,  1. ],
       [ 6.5,  3. ,  5.5,  1.8],
       [ 6.7,  3.3,  5.7,  2.5],
       [ 6. ,  2.2,  5. ,  1.5],
       [ 6.7,  2.5,  5.8,  1.8]])

In [29]:
X_train_2 = np.delete(X_train,1,axis=1)  

In [32]:
X_train_2[0:5]

array([[ 5. ,  3.5,  1. ],
       [ 6.5,  5.5,  1.8],
       [ 6.7,  5.7,  2.5],
       [ 6. ,  5. ,  1.5],
       [ 6.7,  5.8,  1.8]])

In [33]:
X_test_2=np.delete(X_test,1,axis=1)  

In [34]:
forest_2=RandomForestClassifier(n_estimators=10,random_state=0)

In [35]:
forest_2.fit(X_train_2,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [36]:
y_pred_2=forest_2.predict(X_test_2)

In [37]:
confusion_matrix(y_test,y_pred_2)

array([[16,  0,  0],
       [ 0, 17,  1],
       [ 0,  0, 11]])

In [38]:
forest_2.score(X_test_2,y_test)

0.97777777777777775

In [39]:
forest_2.feature_importances_

array([ 0.24582337,  0.46004092,  0.29413571])

从分类结果可以看出去掉  sepal width对分类没有影响 

但有点不理解的是 上一个模型中最重要的特征 现在这个模型中确变成第二重要了 